In [1]:
import torch

if torch.cuda.is_available():
    dev = torch.device('cuda')
    torch.ones(1000, 1000, device=dev)
    print(torch.cuda.get_device_name(dev))
    print('Memory Usage:')
    print('Reserved:', torch.cuda.memory_reserved(dev), 'Bytes')
    print('Allocated:   ', torch.cuda.max_memory_allocated(dev), 'Bytes')

GeForce RTX 2080 Ti
Memory Usage:
Reserved: 20971520 Bytes
Allocated:    4000256 Bytes


In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

A network from the pytorch MNIST example

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
kwargs = {'batch_size': 64, 'num_workers': 1, 'pin_memory': True, 'shuffle': True}

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

dataset1 = datasets.MNIST('./kostya/data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('./kostya/data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

model = Net().to(dev)
optimizer = optim.Adadelta(model.parameters(), lr=1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 4):
    train(model, dev, train_loader, optimizer, epoch)
    test(model, dev, test_loader)
    scheduler.step()

torch.cuda.empty_cache()

Extracting ./kostya/data/MNIST/raw/train-images-idx3-ubyte.gz to ./kostya/data/MNIST/raw


Extracting ./kostya/data/MNIST/raw/train-labels-idx1-ubyte.gz to ./kostya/data/MNIST/raw


Extracting ./kostya/data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./kostya/data/MNIST/raw



Extracting ./kostya/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./kostya/data/MNIST/raw
Processing...
Done!


/root/miniconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729061180/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)








Train Epoch: 1 [0/60000 (0%)]	Loss: 2.286781
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.291153
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.103333
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.273409
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.163127
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.176175
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.034971
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.110043
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.075193
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.161666

Test set: Average loss: 0.0484, Accuracy: 9835/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.143285
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.087208
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.024423
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.116368
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.051084
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.284577
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.174889
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.103815
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.01